In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
import re
from collections import Counter
import liwc
import math
import json
import os
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
import arabic_reviews

In [3]:
books_data = pd.read_csv("appen_translated - books.csv")
books_conf = books_data[(books_data["confidence"] == 1.0) & (books_data["helpfulness"] == "helpful") | (books_data["helpfulness"] != "helpful")]
books_data.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,helpfulness,confidence,no,review,review_en
0,2905523768,False,finalized,3,12/23/2020 7:25:58,somewhat_helpful,0.6598,64963,مهما كانت ميولك الفكريه .. لازم هتحب حاجه في ا...,Whatever the intellectual tastes .. necessary ...
1,2905523769,False,finalized,3,12/23/2020 7:25:58,helpful,0.6598,52854,.. .. هنا التقيت بأحد أعجب الرجال اطلاقا،شمس ا...,.. .. here I met one impressed by the men at a...
2,2905523770,False,finalized,3,12/23/2020 7:25:58,unrelated,0.6598,48057,سأعود,I'll come back
3,2905523771,False,finalized,3,12/23/2020 7:25:58,helpful,1.0000,14611,لم أكن قد قرأتُ لأحلام ثلاثيتها الشهيرة، ولم أ...,I had not read the dreams of the famous Thelat...
4,2905523772,False,finalized,3,12/23/2020 7:54:33,helpful,1.0000,17368,تسحرني لغتها وقلمها. تمنيت أن صفحات الكتاب لا ...,Language fascinates me and her pen. I wished t...


In [4]:
len(books_conf)

1480

In [5]:
len(books_conf[books_conf["helpfulness"] == "helpful"])

818

In [6]:
len(books_conf[books_conf["helpfulness"] != "helpful"])

662

In [7]:
len(books_data)

2404

In [10]:
books_data.groupby("helpfulness").count()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,confidence,no,review,review_en
helpfulness,,,,,,,,,
helpful,1742,1742,1742,1742,1742,1742,1742,1742,1742
not_helpful,170,170,170,170,170,170,170,170,170
somewhat_helpful,410,410,410,410,410,410,410,410,410
unrelated,82,82,82,82,82,82,82,82,82


In [15]:
books_conf.groupby("helpfulness").count()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,confidence,no,review,review_en
helpfulness,,,,,,,,,
helpful,818,818,818,818,818,818,818,818,818
not_helpful,170,170,170,170,170,170,170,170,170
somewhat_helpful,410,410,410,410,410,410,410,410,410
unrelated,82,82,82,82,82,82,82,82,82


In [17]:
(books_data.confidence == 1).sum()

887

In [18]:
len(books_data[(books_data["confidence"] == 1.0) & (books_data["helpfulness"] == "helpful")])

818

In [19]:
hotels_data = pd.read_csv("appen_translated - hotels.csv")
hotels_conf = hotels_data[(hotels_data["confidence"] == 1.0) & (hotels_data["helpfulness"] == "helpful") | (hotels_data["helpfulness"] != "helpful")]
hotels_data.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,helpfulness,confidence,no,review,review_en
0,2892106090,False,finalized,3,12/10/2020 22:54:18,helpful,1.0000,23768,جيد. مديرة خدمة العملاء الانسه بشرى جدا متميزة...,good. Director of Customer Service Miss Bushra...
1,2892106091,False,finalized,3,12/10/2020 22:54:18,helpful,1.0000,173933,مقبول. موقع ممتاز. انتظار الاصنصير طويل لم يكن...,Acceptable. Excellent location. Waiting for a ...
2,2892106092,False,finalized,3,12/10/2020 22:54:18,helpful,0.6765,213748,جيد. الموقع ، طاقم العمل بشوشين ، مكان الافطار...,"good. Location, staff Bchocan, place a small b..."
3,2892106093,False,finalized,3,12/10/2020 22:54:18,helpful,1.0000,147649,“ممتازة”. كل شي رائع و نظيف و الاسرة نظيفة ومر...,"""Excellent"". Everything wonderful and clean an..."
4,2892106094,False,finalized,3,12/10/2020 22:55:10,helpful,0.6604,89990,“اروع فندق”. كل شي كان فندق حقيقة اكثر من رائع...,"""Finest hotel."" Everything was more than a hot..."


In [20]:
classes_conf = hotels_conf["helpfulness"].map(lambda x: "not_helpful" if x != "helpful" else "helpful")

In [21]:
len(hotels_conf)

1624

In [22]:
len(hotels_conf[hotels_conf["helpfulness"] == "helpful"])

983

In [23]:
len(hotels_conf[hotels_conf["helpfulness"] != "helpful"])

641

In [25]:
hotels_data.groupby("helpfulness").count()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,confidence,no,review,review_en
helpfulness,,,,,,,,,
helpful,2143,2143,2143,2143,2143,2143,2143,2143,2143
not_helpful,135,135,135,135,135,135,135,135,135
somewhat_helpful,475,475,475,475,475,475,475,475,475
unrelated,31,31,31,31,31,31,31,31,31


In [26]:
data = pd.read_json("EnglishBookReviews.json")

In [27]:
len(data)

330000

In [28]:
books_data = pd.read_csv("Book_labeled_all_rows_with_ratings.csv")
books_data.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,helpfulness,confidence,helpfulness_gold,no,review,rating,ratings_combined
0,2905523768,False,finalized,3,12/23/2020 07:25:58,somewhat_helpful,0.6598,NaN,64963,مهما كانت ميولك الفكريه .. لازم هتحب حاجه في ا...,5.0,pos_rating
1,2905523769,False,finalized,3,12/23/2020 07:25:58,helpful,0.6598,NaN,52854,.. .. هنا التقيت بأحد أعجب الرجال اطلاقا،شمس ا...,5.0,pos_rating
2,2905523770,False,finalized,3,12/23/2020 07:25:58,unrelated,0.6598,NaN,48057,سأعود,3.0,neutral_ratings
3,2905523771,False,finalized,3,12/23/2020 07:25:58,helpful,1.0000,NaN,14611,لم أكن قد قرأتُ لأحلام ثلاثيتها الشهيرة، ولم أ...,2.0,neg_rating
4,2905523772,False,finalized,3,12/23/2020 07:54:33,helpful,1.0000,NaN,17368,تسحرني لغتها وقلمها. تمنيت أن صفحات الكتاب لا ...,4.0,pos_rating


In [29]:
len(books_data)

2404

In [30]:
books_data.groupby("rating").count()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,helpfulness,confidence,helpfulness_gold,no,review,ratings_combined
rating,,,,,,,,,,,
1.0,156,156,156,156,156,156,156,0,156,156,156
2.0,201,201,201,201,201,201,201,0,201,201,201
3.0,412,412,412,412,412,412,412,0,412,412,412
4.0,715,715,715,715,715,715,715,0,715,715,715
5.0,920,920,920,920,920,920,920,0,920,920,920
